In [33]:
%matplotlib inline
import numpy as np
from keras.datasets import imdb
from keras.layers import Flatten, \
    Dense, \
    Embedding, \
    BatchNormalization, \
    Dropout, \
    Input, \
    Convolution1D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
import pickle

## Prepare Dataset

### Load Word Index

In [2]:
word_index = imdb.get_word_index()
idx_word = {idx:word for word, idx in word_index.iteritems()}

### Load IMDB Dataset

In [25]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

#### Sample of first 100 words

In [39]:
' '.join([idx_word[i] for i in x_train[0][0:100]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high"

### Data munging for training

In [61]:
vocab_size = 5000
n_factors = 50
review_length = 500

In [48]:
def massage_data(review_list):
    return [np.minimum(np.array(review), vocab_size - 1) for review in review_list]

In [50]:
trn = massage_data(x_train)
val = massage_data(x_test)

In [69]:
trn = pad_sequences(trn, maxlen=review_length)
val = pad_sequences(val, maxlen=review_length)

## Create Basic Model

In [109]:
model = Sequential([Embedding(input_dim=vocab_size, output_dim=n_factors, input_length=review_length),
                    Dense(32, activation='relu'),
                    Flatten(),
                    Dropout(0.7),
                    Dense(1, activation='sigmoid')])

In [113]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_10 (Embedding)         (None, 500, 50)       250000      embedding_input_8[0][0]          
____________________________________________________________________________________________________
dense_16 (Dense)                 (None, 500, 32)       1632        embedding_10[0][0]               
____________________________________________________________________________________________________
flatten_9 (Flatten)              (None, 16000)         0           dense_16[0][0]                   
____________________________________________________________________________________________________
dropout_6 (Dropout)              (None, 16000)         0           flatten_9[0][0]                  
___________________________________________________________________________________________

In [110]:
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [111]:
model.fit(trn, labels_train, batch_size=64, nb_epoch=1, validation_data=(val, labels_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 5s - loss: 0.5639 - acc: 0.6734 - val_loss: 0.3104 - val_acc: 0.8729


In [112]:
model.optimizer.lr = 0.1
model.fit(trn, labels_train, batch_size=64, nb_epoch=1, validation_data=(val, labels_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 5s - loss: 0.2872 - acc: 0.8808 - val_loss: 0.2792 - val_acc: 0.8834


## Single Conv Layer

In [135]:
model = Sequential([Embedding(input_dim=vocab_size, output_dim=n_factors, input_length=review_length),
                    Convolution1D(nb_filter=32, filter_length=3),
                    Dropout(0.2),
                    Dense(100, activation='relu'),
                    BatchNormalization(),
                    Flatten(),
                    Dropout(0.7),
                    Dense(1, activation='sigmoid')])

In [136]:
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [137]:
model.fit(trn, labels_train, batch_size=128, nb_epoch=1, validation_data=(val, labels_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 10s - loss: 0.4754 - acc: 0.7427 - val_loss: 0.6557 - val_acc: 0.5087


In [140]:
model.optimizer.lr = 0.1
model.fit(trn, labels_train, batch_size=128, nb_epoch=1, validation_data=(val, labels_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 9s - loss: 0.1942 - acc: 0.9243 - val_loss: 0.4133 - val_acc: 0.8311
